# Author: Umang Srivastav

Here ResNet-50 is trained on CIFAR10

In [ ]:
import torch
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torchvision.models as models

transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5071, 0.4865, 0.4409],
                         std=[0.2673, 0.2564, 0.2762])
])

train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=128, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=128, shuffle=False)

model = models.resnet50(pretrained=False, num_classes=100)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.1, momentum=0.9, weight_decay=1e-4)

num_epochs = 100
for epoch in range(num_epochs):
    
    for images, labels in train_loader:
        
        if torch.cuda.is_available():
            images = images.cuda()
            labels = labels.cuda()

        optimizer.zero_grad()
        
        outputs = model(images)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in test_loader:
            if torch.cuda.is_available():
                images = images.cuda()
                labels = labels.cuda()

            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        accuracy = 100 * correct / total
        print(f"Epoch [{epoch + 1}/{num_epochs}], Test Accuracy: {accuracy:.2f}%")


100%|██████████| 170498071/170498071 [00:01<00:00, 103017669.11it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Epoch [1/100], Test Accuracy: 19.34%
Epoch [2/100], Test Accuracy: 28.57%
Epoch [3/100], Test Accuracy: 32.79%
Epoch [4/100], Test Accuracy: 38.72%
Epoch [5/100], Test Accuracy: 42.02%
Epoch [6/100], Test Accuracy: 43.44%
Epoch [7/100], Test Accuracy: 45.46%
Epoch [8/100], Test Accuracy: 46.05%
Epoch [9/100], Test Accuracy: 48.43%
Epoch [10/100], Test Accuracy: 50.62%
Epoch [11/100], Test Accuracy: 47.17%
Epoch [12/100], Test Accuracy: 52.64%
Epoch [13/100], Test Accuracy: 54.08%
Epoch [14/100], Test Accuracy: 53.84%
Epoch [15/100], Test Accuracy: 55.77%
Epoch [16/100], Test Accuracy: 56.77%
Epoch [17/100], Test Accuracy: 57.34%
Epoch [18/100], Test Accuracy: 58.58%
Epoch [19/100], Test Accuracy: 59.40%
Epoch [20/100], Test Accuracy: 61.93%
Epoch [21/100], Test Accuracy: 62.28%
Epoch [22/100], Test Accuracy: 58.75%
Epoch [23/100], Test Accuracy: 63.21%
Epoch [24/100], Test Accuracy: 65.11%
Epoch [25/100], Test Accuracy: 66.16%
Epoch [26/100], Test Accuracy: 64.86%
Epoch [27/100], Test 

In [ ]:
torch.save(model.state_dict(), 'model50.pt') #Saving the model

In [ ]:
import torch
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torchvision.models as models

model=models.resnet50(pretrained=False, num_classes=100)
model.load_state_dict(torch.load('model50.pt'))

<All keys matched successfully>

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

Here I have taken a first few layers of trained model and fine tuned with adding 2 extra FC layers and trained on CIFAR100.

In [ ]:
import torch
import torchvision.models as models

class ModifiedResNet50(torch.nn.Module):
    def __init__(self, model):
        super().__init__()
        
        resnet=model#.resnet
        self.conv1=resnet.conv1
        self.bn1 = resnet.bn1
        self.relu = resnet.relu
        self.maxpool = resnet.maxpool
        self.layer1 = resnet.layer1
        self.layer2 = resnet.layer2
        # del self.layer2[3]
        # del self.layer2[2].conv2
        # del self.layer2[2].conv3
        # del self.layer2[2].bn2
        # del self.layer2[2].bn3
        self.avgpool = resnet.avgpool
        self.fc=torch.nn.Linear(in_features=512, out_features=100, bias=True)

        self.fc2=torch.nn.Linear(in_features=100, out_features=500, bias=True)
        self.fc3=torch.nn.Linear(in_features=500, out_features=100, bias=True)
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)

        x = self.fc2(x)
        x = self.fc3(x)
        
        return x


model = ModifiedResNet50(model) 
model=model.to(device)
# print(model)
import torchvision
for name, param in model.named_parameters():
    if name.startswith('fc'):
        param.requires_grad = True
    else:
        param.requires_grad = False

for name, param in model.named_parameters():
    if param.requires_grad:
        print(name, "is trainable")
    else:
        print(name, "is not trainable")

transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5071, 0.4865, 0.4409],
                         std=[0.2673, 0.2564, 0.2762])
])


trainset = torchvision.datasets.CIFAR100(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=32, shuffle=True, num_workers=2)


validset = torchvision.datasets.CIFAR100(root='./data', train=False, download=True, transform=transform)
validloader = torch.utils.data.DataLoader(validset, batch_size=32, shuffle=False, num_workers=2)

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

num_epochs = 100
for epoch in range(num_epochs):
    
    for images, labels in trainloader:
        
        if torch.cuda.is_available():
            images = images.cuda()
            labels = labels.cuda()

        optimizer.zero_grad()
        
        outputs = model(images)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in validloader:
            if torch.cuda.is_available():
                images = images.cuda()
                labels = labels.cuda()

            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        accuracy = 100 * correct / total
        print(f"Epoch [{epoch + 1}/{num_epochs}], Test Accuracy: {accuracy:.2f}%")



torch.save(model.state_dict(), 'Modmodel50-18.pt')

conv1.weight is not trainable
bn1.weight is not trainable
bn1.bias is not trainable
layer1.0.conv1.weight is not trainable
layer1.0.bn1.weight is not trainable
layer1.0.bn1.bias is not trainable
layer1.0.conv2.weight is not trainable
layer1.0.bn2.weight is not trainable
layer1.0.bn2.bias is not trainable
layer1.0.conv3.weight is not trainable
layer1.0.bn3.weight is not trainable
layer1.0.bn3.bias is not trainable
layer1.0.downsample.0.weight is not trainable
layer1.0.downsample.1.weight is not trainable
layer1.0.downsample.1.bias is not trainable
layer1.1.conv1.weight is not trainable
layer1.1.bn1.weight is not trainable
layer1.1.bn1.bias is not trainable
layer1.1.conv2.weight is not trainable
layer1.1.bn2.weight is not trainable
layer1.1.bn2.bias is not trainable
layer1.1.conv3.weight is not trainable
layer1.1.bn3.weight is not trainable
layer1.1.bn3.bias is not trainable
layer1.2.conv1.weight is not trainable
layer1.2.bn1.weight is not trainable
layer1.2.bn1.bias is not trainable
lay

Here I have taken a first few layers of trained model and fine tuned with adding an FC layers and trained on CIFAR10.

In [ ]:
import torch
import torchvision.models as models

class ModifiedResNet50(torch.nn.Module):
    def __init__(self, model):
        super().__init__()
        
        resnet=model#.resnet
        self.conv1=resnet.conv1
        self.bn1 = resnet.bn1
        self.relu = resnet.relu
        self.maxpool = resnet.maxpool
        self.layer1 = resnet.layer1
        self.layer2 = resnet.layer2
        # del self.layer2[3]
        # del self.layer2[2].conv2
        # del self.layer2[2].conv3
        # del self.layer2[2].bn2
        # del self.layer2[2].bn3
        self.avgpool = resnet.avgpool
        self.fc=torch.nn.Linear(in_features=512, out_features=10, bias=True)

        # self.fc2=torch.nn.Linear(in_features=100, out_features=500, bias=True)
        # self.fc3=torch.nn.Linear(in_features=500, out_features=100, bias=True)
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)

        # x = self.fc2(x)
        # x = self.fc3(x)
        
        return x


model = ModifiedResNet50(model) 
model=model.to(device)
# print(model)
import torchvision
for name, param in model.named_parameters():
    if name.startswith('fc'):
        param.requires_grad = True
    else:
        param.requires_grad = False

for name, param in model.named_parameters():
    if param.requires_grad:
        print(name, "is trainable")
    else:
        print(name, "is not trainable")

transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5071, 0.4865, 0.4409],
                         std=[0.2673, 0.2564, 0.2762])
])


trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=32, shuffle=True, num_workers=2)


validset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
validloader = torch.utils.data.DataLoader(validset, batch_size=32, shuffle=False, num_workers=2)

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

num_epochs = 100
for epoch in range(num_epochs):
    
    for images, labels in trainloader:
        
        if torch.cuda.is_available():
            images = images.cuda()
            labels = labels.cuda()

        optimizer.zero_grad()
        
        outputs = model(images)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in validloader:
            if torch.cuda.is_available():
                images = images.cuda()
                labels = labels.cuda()

            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        accuracy = 100 * correct / total
        print(f"Epoch [{epoch + 1}/{num_epochs}], Test Accuracy: {accuracy:.2f}%")



torch.save(model.state_dict(), 'Modmodel50-18.pt')

conv1.weight is not trainable
bn1.weight is not trainable
bn1.bias is not trainable
layer1.0.conv1.weight is not trainable
layer1.0.bn1.weight is not trainable
layer1.0.bn1.bias is not trainable
layer1.0.conv2.weight is not trainable
layer1.0.bn2.weight is not trainable
layer1.0.bn2.bias is not trainable
layer1.0.conv3.weight is not trainable
layer1.0.bn3.weight is not trainable
layer1.0.bn3.bias is not trainable
layer1.0.downsample.0.weight is not trainable
layer1.0.downsample.1.weight is not trainable
layer1.0.downsample.1.bias is not trainable
layer1.1.conv1.weight is not trainable
layer1.1.bn1.weight is not trainable
layer1.1.bn1.bias is not trainable
layer1.1.conv2.weight is not trainable
layer1.1.bn2.weight is not trainable
layer1.1.bn2.bias is not trainable
layer1.1.conv3.weight is not trainable
layer1.1.bn3.weight is not trainable
layer1.1.bn3.bias is not trainable
layer1.2.conv1.weight is not trainable
layer1.2.bn1.weight is not trainable
layer1.2.bn1.bias is not trainable
lay